In [0]:
dbutils.library.restartPython()

In [0]:

%sh

pip install azure-identity
pip install azure-monitor-query

  Obtaining dependency information for azure-identity from https://files.pythonhosted.org/packages/3d/9f/1f9f3ef4f49729ee207a712a5971a9ca747f2ca47d9cbf13cf6953e3478a/azure_identity-1.21.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.8 MB/s eta 0:00:00
  Obtaining dependency information for azure-core>=1.31.0 from https://files.pythonhosted.org/packages/07/b7/76b7e144aa53bd206bf1ce34fa75350472c3f69bf30e5c8c18bc9881035d/azure_core-1.33.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.3 MB/s eta 0:00:00
  Obtaining dependency information for msal>=1.30.0 from https://files.pythonhosted.org/packages/04/bf/81516b9aac7fd867709984d08eb4db1d2e3fe1df795c8e442cde9b568962/msal-1.32.3-py3-none-any.whl.metadata
  Obtaining dependency information for msal-extensions>=1.2.0 from https://files.pythonhosted.org/packages/5e/75/bd9b7bb966668920f06b200e84454c8f3566b102183bc55c5473d96cb2b9/msal_extensions-1.3.1-py3-none-any.w


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


  Obtaining dependency information for azure-monitor-query from https://files.pythonhosted.org/packages/cb/71/bdf03aef699d9a17d80bcaa8255c20e596ed591daa8a42a21854ddc4277d/azure_monitor_query-1.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 9.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: pip install --upgrade pip



### doc reference 

https://learn.microsoft.com/en-us/azure/network-watcher/traffic-analytics-schema?tabs=vnet#traffic-analytics-schema

In [0]:
vnet_logs = '''

NTANetAnalytics
| where FlowType != '' and SubType == 'FlowLog'
| extend SrcPIP = substring(SrcPublicIps, 0, indexof(SrcPublicIps, "|"))
| extend DestPIP = substring(DestPublicIps, 0, indexof(DestPublicIps, "|"))

| extend SrcSubnetSplitted =  split(SrcSubnet, '/')
| extend SrcRG = tostring(SrcSubnetSplitted[0])
| extend SrcVNet = tostring(SrcSubnetSplitted[1])
| extend SrcSubnetName = tostring(SrcSubnetSplitted[2])

//check "location" of src node
| extend SrcNodeInOrOutVNet =   iif(SrcIp != '' and FlowType != 'MaliciousFlow', 'NodeInsideVNet',
                                  iif(FlowType == 'AzurePublic' and SrcPublicIps != '', 'NodeOutsideVNet_PaaS', 
                                    iif(FlowType == 'ExternalPublic' and SrcPublicIps != '', 'NodeOutsideAzure',
                                      //adversary from outside of Azure 
                                      iif (FlowType =='MaliciousFlow' and FlowDirection == 'Inbound' and SrcIp != '', 'NodeMalicious',
                                        //adversary from inside of Azure 
                                        iif(FlowType =='MaliciousFlow' and FlowDirection == 'Outbound', 'NodeInsideVNet', 'NodeUnknown')
                                )
                              )
                            )
                          )


| extend DestSubnetSplitted =  split(DestSubnet, '/')
| extend DestRG = tostring(DestSubnetSplitted[0])
| extend DestVNet = tostring(DestSubnetSplitted[1])
| extend DestSubnetName = tostring(DestSubnetSplitted[2])

//check "location" of dest node
| extend DestNodeInOrOutVNet = iif(DestIp != '' and FlowType != 'MaliciousFlow', 'NodeInsideVNet',
                            iif(FlowType in ('AzurePublic') and DestPublicIps != '', 'NodeOutsideVNet_PaaS', 
                                iif(FlowType == 'ExternalPublic' and DestPublicIps != '', 'NodeOutsideAzure',
                                    //adversary from outside of Azure 
                                    iif(FlowType == 'MaliciousFlow' and FlowDirection== 'Inbound' and DestIp != '', 'NodeInsideVNet',
                                        //adversary from inside of Azure 
                                        iif(FlowType == 'MaliciousFlow' and FlowDirection== 'Outbound', 'NodeMalicious', 'NodeUnknown')
                                )
                              )
                            )
                          )

| join kind=leftouter 
(NTAIpDetails 
| project ['SrcPIP_Ip']=Ip, ['SrcPIP_PublicIpDetails']=PublicIpDetails, ['SrcPIP_ThreatType']=ThreatType, ['SrcPIP_Location']=Location, ['SrcPIP_Url']=Url, Src_NTAIpDetails=1
)
on $left.SrcPIP == $right.SrcPIP_Ip

| join kind=leftouter
    (
        NTAIpDetails 
        | project ['DestPIP_Ip']=Ip, ['DestPIP_PublicIpDetails']=PublicIpDetails, ['DestPIP_ThreatType']=ThreatType, ['DestPIP_Location']=Location, ['DestPIP_Url']=Url, Dest_NTAIpDetails=1
    )
on $left.DestPIP == $right.DestPIP_Ip

// set SrcPIP_Ip if exist
| extend SrcIp = iif(SrcPIP_Ip != '', SrcPIP_Ip, SrcIp)

// resolve SrcName as private endpoint name if traffic is private endpoint
// *note: private endpoint is Source and not destination
| extend PrivateEndpointName = split(PrivateEndpointResourceId, '/')[-1]
| extend PrivateLinkName = split(PrivateEndpointResourceId, '/')[-2]

| extend SrcName = iif(SrcApplicationGateway != '', SrcApplicationGateway, 
                     iif(SrcLoadBalancer != '', SrcLoadBalancer, 
                        iif(SrcPIP_PublicIpDetails != '', SrcPIP_PublicIpDetails, 
                          iif(SrcSubnetName == 'azurebastionsubnet', 'bastion-vm',
                           iif(PrivateEndpointResourceId != '' and PrivateLinkResourceId != '',  PrivateEndpointName, 
                             iif(SrcNic startswith 'unknown', strcat('managed vm in ', iif(SrcSubnetName has 'subnet', SrcSubnetName, strcat(SrcSubnetName, ' subnet'))), 
                               iif(SrcVm != '', SrcVm, '-' )))))))

| extend SrcName = iif(indexof(SrcName, '/',0) > 0, split(SrcName, '/')[-1], SrcName)

// set DestPIP_Ip if exist
| extend DestIp = strcat(iif(DestPIP_Ip != '', DestPIP_Ip, DestIp), ':', DestPort)

| extend DestName = iif(DestApplicationGateway != '', DestApplicationGateway, 
                        iif(DestLoadBalancer != '', DestLoadBalancer,
                            iif(DestPIP_PublicIpDetails != '', DestPIP_PublicIpDetails,
                               iif(DestSubnetName == 'azurebastionsubnet', 'bastion-vm', 
                                iif(PrivateEndpointResourceId != '' and PrivateLinkResourceId != '',  PrivateLinkName, 
                                  iif(DestNic startswith 'unknown', strcat('managed vm in ', iif(DestSubnetName has 'subnet', DestSubnetName, strcat(DestSubnetName, ' subnet'))), 
                                    iif(DestVm != '',  DestVm, '' )))))))


| extend DestName = iif(indexof(DestName, '/',0) > 0, split(DestName, '/')[-1], DestName)


| extend protocol = tolower(strcat(L4Protocol,  iif(L7Protocol != 'Unknown', strcat(':', L7Protocol), ''))) 

| summarize BytesSrcToDest = max(BytesSrcToDest), BytesDestToSrc = max(BytesDestToSrc)
            by Src_NTAIpDetails, Dest_NTAIpDetails, FlowType, FlowDirection, ConnectionType, protocol,
                    SrcNodeInOrOutVNet, SrcSubscription, SrcRG, SrcVNet, SrcSubnetName , SrcIp, SrcName,
                    DestNodeInOrOutVNet, DestSubscription, DestRG, DestVNet, DestSubnetName, DestName,
                    SrcPIP_ThreatType, SrcPIP_Location, SrcPIP_Url,
                    DestIp, DestPIP_ThreatType, DestPIP_Location, DestPIP_Url


| extend SrcToDestDataSize = format_bytes(BytesSrcToDest, 2)
| extend DestToSrcDataSize = format_bytes(BytesDestToSrc, 2)  


| where SrcIp != '' or DestIp != ''

//SrcName and DestName can be 
    // VM
    // App Gateway
    // Load Balancer
    // Private Endpoint
    // PaaS services
    // external endpoints which has no name and is empty
    // on-premise which has no name and is empty
| distinct Src_NTAIpDetails, Dest_NTAIpDetails, FlowType, FlowDirection, ConnectionType, Protocol=protocol,
        SrcNodeInOrOutVNet, SrcSubscription, SrcRG, SrcVNet, SrcSubnetName , SrcIp, SrcName, SrcToDestDataSize,
        DestNodeInOrOutVNet, DestSubscription, DestRG, DestVNet, DestSubnetName, DestName, DestToSrcDataSize,
        SrcPIP_ThreatType, SrcPIP_Location, SrcPIP_Url,
        DestIp, DestPIP_ThreatType, DestPIP_Location, DestPIP_Url

'''

In [0]:
from azure.monitor.query import LogsQueryClient
from azure.identity import DefaultAzureCredential
import datetime
from datetime import timedelta
import pandas as pd
import math
import json

credential = DefaultAzureCredential()
client = LogsQueryClient(credential)

now = datetime.datetime.now()
mins_ago = now - timedelta(minutes=30)

response = client.query_workspace(
            workspace_id= 'c596239e-beda-4999-a79f-58e34d9881e4',
            query=vnet_logs,
            timespan=timedelta(hours=2)
        )

table = response.tables[0]

df = pd.DataFrame(data=table.rows, columns=table.columns)


In [0]:
print(df['FlowType'].unique())

['AzurePublic' 'ExternalPublic' 'MaliciousFlow' 'UnknownPrivate'
 'IntraVNet' 'InterVNet']


In [0]:

# drop all logs where they are: Inbound-ExternalPublic and found in NTAIpDetails table
externalpublic_inbound_NTAIpDetails_match_indexes = df[(df['FlowDirection']=='Inbound') & (df['FlowType']=='ExternalPublic') & (df['SrcName'] != '')].index

tempdf = df.drop(externalpublic_inbound_NTAIpDetails_match_indexes)

# update SrcName by Subnet
                                               
tempdf['SrcName'] = tempdf.apply(lambda x: x['SrcSubnetName'] if x['SrcName']=='' else x['SrcName'], axis=1)

maindf = tempdf.drop_duplicates(subset=['SrcName', 'DestName'])

maindf.fillna('', inplace=True)

print(len(maindf.index))

maindf





105


/root/.ipykernel/887/command-5477959247676144-3026362402:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maindf.fillna('', inplace=True)


,Src_NTAIpDetails,Dest_NTAIpDetails,FlowType,FlowDirection,ConnectionType,Protocol,SrcNodeInOrOutVNet,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestNodeInOrOutVNet,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
0,,1.0,AzurePublic,Outbound,,tcp:https,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,azurebastionsubnet,172.18.0.68,bastion-vm,50.24 KB,NodeOutsideVNet_PaaS,,,,,AzureFrontDoor.Frontend,4.07 MB,,,,13.107.246.59:443,,,
1,,1.0,AzurePublic,Outbound,,tcp:https,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.5,vnetgw-cwhd-onpremise-VM-192.166.0.5,4.21 KB,NodeOutsideVNet_PaaS,,,,,AzureFrontDoor.Frontend,18.49 KB,,,,13.107.246.59:443,,,
3,,1.0,AzurePublic,Outbound,,tcp:entextxid,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,172.18.0.4,vpngw-cwhd-s2s-azure-VM-172.18.0.4,1.2 KB,NodeOutsideVNet_PaaS,,,,,AzureMonitor,1.25 KB,,,,13.69.233.97:12000,,northeurope,
4,,1.0,AzurePublic,Outbound,,tcp:entextxid,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-ingress,apim,172.16.2.5,managed vm in apim subnet,2.52 KB,NodeOutsideVNet_PaaS,,,,,AzureMonitor,1.72 KB,,,,13.69.233.97:12000,,northeurope,
5,,1.0,AzurePublic,Outbound,,tcp:entextxid,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,172.18.0.5,vpngw-cwhd-s2s-azure-VM-172.18.0.5,1.52 KB,NodeOutsideVNet_PaaS,,,,,AzureMonitor,1.25 KB,,,,13.69.233.97:12000,,northeurope,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,1.0,,AzurePublic,Inbound,,tcp,NodeOutsideVNet_PaaS,,,,,40.87.192.0,AzureCloud,128 Bytes,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd,vm,vm-inter-vm-traffic,0 Bytes,,eastasia,,13.0.2.4:23390,,,
367,1.0,,AzurePublic,Inbound,,tcp,NodeOutsideVNet_PaaS,,,,,40.87.192.0,AzureCloud,128 Bytes,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-management,vms,vm-build-agent-win-server-1,0 Bytes,,eastasia,,10.0.1.75:25816,,,
378,1.0,,AzurePublic,Inbound,,tcp,NodeOutsideVNet_PaaS,,,,,40.87.192.0,AzureCloud,128 Bytes,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,vm,vm-cwhd-s2s-azure,0 Bytes,,eastasia,,172.18.1.4:24605,,,
610,1.0,,AzurePublic,Inbound,,udp:isakmp,NodeOutsideVNet_PaaS,,,,,20.62.198.225,AzureCloud,510 Bytes,NodeInsideVNet,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,vpngw-cwhd-s2s-azure-VM-52.163.205.184,0 Bytes,,eastus,,52.163.205.184:500,,,


In [0]:
    # {
    #   name: 'CNF',
    #   id: 'CNF',
    #   category: 1,
    #   ip: '12.20.11.23',
    #   subnet: "some text",
    #   vnet: '123'
    # },



# create nodes csv for IntraVNet

src_nodes_final_df = pd.DataFrame()
src_nodes_final_df['id'] = maindf['SrcIp']
src_nodes_final_df['name'] = maindf['SrcName']
src_nodes_final_df['category'] = maindf['FlowType']
src_nodes_final_df['ip'] = maindf['SrcIp']
src_nodes_final_df['subnet'] = maindf['SrcSubnetName']
src_nodes_final_df['vnet'] = maindf['SrcVNet']
src_nodes_final_df['srcPIPLocation'] = maindf['SrcPIP_Location']


dest_nodes_final_df = pd.DataFrame()
dest_nodes_final_df['id'] = maindf['DestIp']
dest_nodes_final_df['name'] = maindf['DestName']
dest_nodes_final_df['category'] = maindf['FlowType']
dest_nodes_final_df['ip'] = maindf['DestIp']
dest_nodes_final_df['subnet'] = maindf['DestSubnetName']
dest_nodes_final_df['vnet'] = maindf['DestVNet']
src_nodes_final_df['destPIPLocation'] = maindf['DestPIP_Location']

# # #id,title,subtitle,mainstat,secondarystat,color
combined_list = [src_nodes_final_df, dest_nodes_final_df]

final_nodes_df = pd.concat(combined_list)

final_nodes_df = final_nodes_df.drop_duplicates(subset=['id'])

final_nodes_json_str =  final_nodes_df.to_json(orient='records')

final_nodes_json_obj = json.loads(final_nodes_json_str)


print(final_nodes_json_str)


[{"id":"172.18.0.68","name":"bastion-vm","category":"AzurePublic","ip":"172.18.0.68","subnet":"azurebastionsubnet","vnet":"vnet-cwhd-s2s-azure","srcPIPLocation":"","destPIPLocation":""},{"id":"192.166.0.5","name":"vnetgw-cwhd-onpremise-VM-192.166.0.5","category":"AzurePublic","ip":"192.166.0.5","subnet":"gatewaysubnet","vnet":"vnet-cwh-s2s-onpremise","srcPIPLocation":"","destPIPLocation":""},{"id":"172.18.0.4","name":"vpngw-cwhd-s2s-azure-VM-172.18.0.4","category":"AzurePublic","ip":"172.18.0.4","subnet":"gatewaysubnet","vnet":"vnet-cwhd-s2s-azure","srcPIPLocation":"","destPIPLocation":"northeurope"},{"id":"172.16.2.5","name":"managed vm in apim subnet","category":"AzurePublic","ip":"172.16.2.5","subnet":"apim","vnet":"vnet-ingress","srcPIPLocation":"","destPIPLocation":"northeurope"},{"id":"172.18.0.5","name":"vpngw-cwhd-s2s-azure-VM-172.18.0.5","category":"AzurePublic","ip":"172.18.0.5","subnet":"gatewaysubnet","vnet":"vnet-cwhd-s2s-azure","srcPIPLocation":"","destPIPLocation":"north

In [0]:

    # {
    #   source: 'CNF',
    #   target: 'SCPC',
    #   src_to_dest_data_size: '10 KB',
    #   dest_to_srct_data_size: '22 KB',
    #   flowType: 'Azure Public PaaS',
    #   label: {
    #     show: true,
    #     formatter: (params) => params.data.src_to_dest_data_size + '-> <- ' + params.data.dest_to_srct_data_size + '\n' + params.data.flowType,
    #     fontSize: 10
    #   },
    #   lineStyle: {
    #     curveness: 0.1
    #   }
    # }

# create edge

edges_df = pd.DataFrame()
edges_df['source'] = tempdf['SrcIp']
edges_df['target'] = tempdf['DestIp']
# edges_df['value'] = tempdf['SrcToDestDataSize'] + '-> <-' + tempdf['DestToSrcDataSize'] + '<div> flowtype: ' + tempdf['FlowType'] + '</div>' + '<div> protocol: ' +  tempdf['protocol'] + '/div>'
edges_df['src_to_dest_data_size'] = tempdf['SrcToDestDataSize']
edges_df['dest_to_srct_data_size'] = tempdf['DestToSrcDataSize']
edges_df['flowType'] = tempdf['FlowType']
edges_df['protocol'] = tempdf['Protocol']
edges_df['connectionType'] = tempdf['ConnectionType']

edges_df = edges_df.drop_duplicates(subset=['source', 'target'])

final_edges_json_str =  edges_df.to_json(orient='records')

final_edges_json_obj = json.loads(final_edges_json_str)

print(final_edges_json_str)


[{"source":"172.18.0.68","target":"13.107.246.59:443","src_to_dest_data_size":"50.24 KB","dest_to_srct_data_size":"4.07 MB","flowType":"AzurePublic","protocol":"tcp:https","connectionType":""},{"source":"192.166.0.5","target":"13.107.246.59:443","src_to_dest_data_size":"4.21 KB","dest_to_srct_data_size":"18.49 KB","flowType":"AzurePublic","protocol":"tcp:https","connectionType":""},{"source":"10.0.1.4","target":"13.107.246.59:443","src_to_dest_data_size":"55.47 KB","dest_to_srct_data_size":"4.07 MB","flowType":"AzurePublic","protocol":"tcp:https","connectionType":""},{"source":"172.18.0.4","target":"13.69.233.97:12000","src_to_dest_data_size":"1.2 KB","dest_to_srct_data_size":"1.25 KB","flowType":"AzurePublic","protocol":"tcp:entextxid","connectionType":""},{"source":"172.16.2.5","target":"13.69.233.97:12000","src_to_dest_data_size":"2.52 KB","dest_to_srct_data_size":"1.72 KB","flowType":"AzurePublic","protocol":"tcp:entextxid","connectionType":""},{"source":"172.18.0.5","target":"13.6

### categories result

In [0]:

# category

# unique_src_node_location = maindf['SrcNodeInOrOutVNet'].unique()
# unique_dest_node_location = maindf['DestNodeInOrOutVNet'].unique()

# unique_categories = list(set(unique_src_node_location) | set(unique_dest_node_location))

# result = pd.DataFrame()
# result['name'] = unique_categories


result = pd.DataFrame()
result['name'] = maindf['FlowType'].unique()


# for x in unique_src_node_location + unique_dest_node_location:
#     result.add(x)

# result = list(result)
unique_categories =  result.to_json(orient='records')

print(unique_categories)

[{"name":"AzurePublic"},{"name":"ExternalPublic"},{"name":"MaliciousFlow"},{"name":"UnknownPrivate"},{"name":"IntraVNet"},{"name":"InterVNet"}]


### node and edges final result

In [0]:
final_json = {
    "nodes": final_nodes_json_obj,
    "edges": final_edges_json_obj,
    "categories": unique_categories
}

json_str = json.dumps(final_json)

with open('nodes_edges.txt', "w") as f:
    f.write(json_str)